In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the dataset (assuming it is available in TensorFlow Datasets)
(train_ds, val_ds), ds_info = tfds.load(
    'stanford_dogs',
    split=['train', 'test'],
    with_info=True,
    as_supervised=True
)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/12000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incompleteZG8W6G/stanford_dogs-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/8580 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incompleteZG8W6G/stanford_dogs-test.tfrecord*...:   0%…

Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


In [ ]:
# Function to preprocess images
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224])  # ResNet50 standard input size
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

# Apply preprocessing to dataset
train_ds = train_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Define the model architecture
base_model = ResNet50(weights='imagenet', include_top=False)
base_model.trainable = False  # Freeze the convolutional base

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
# Train the model and save the history
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Access the history to retrieve training and validation accuracy
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Print final epoch's accuracy directly
print(f"Final Training Accuracy: {train_acc[-1]:.4f}")
print(f"Final Validation Accuracy: {val_acc[-1]:.4f}")

Epoch 1/10
375/375 [==============================] - 85s 207ms/step - loss: 2.5629 - accuracy: 0.4408 - val_loss: 1.2021 - val_accuracy: 0.6888
Epoch 2/10
375/375 [==============================] - 74s 198ms/step - loss: 0.8813 - accuracy: 0.7647 - val_loss: 0.8754 - val_accuracy: 0.7480
Epoch 3/10
375/375 [==============================] - 76s 202ms/step - loss: 0.5994 - accuracy: 0.8387 - val_loss: 0.7859 - val_accuracy: 0.7681
Epoch 4/10
375/375 [==============================] - 74s 198ms/step - loss: 0.4551 - accuracy: 0.8840 - val_loss: 0.7490 - val_accuracy: 0.7752
Epoch 5/10
375/375 [==============================] - 85s 226ms/step - loss: 0.3579 - accuracy: 0.9163 - val_loss: 0.7324 - val_accuracy: 0.7780
Epoch 6/10
375/375 [==============================] - 86s 228ms/step - loss: 0.2861 - accuracy: 0.9417 - val_loss: 0.7251 - val_accuracy: 0.7801
Epoch 7/10
375/375 [==============================] - 74s 198ms/step - loss: 0.2309 - accuracy: 0.9592 - val_loss: 0.7222 - val_ac